In [1]:
from modelsDenseFeatures import *

In [2]:
df = pd.read_csv("torchMoji-master/examples/emotions_dataset_class").fillna(' ')

In [3]:
df.head()

,E0,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E183,E184,E185,E186,E187,E188,E189,E190,E191,class
0,0.003607,0.021549,0.024439,0.027935,0.000726,0.066190,0.002535,0.002217,0.003601,0.002111,...,0.001190,0.002276,0.000609,0.000355,0.001201,0.000567,0.001560,0.003496,0.002619,3
1,0.037200,0.014853,0.007491,0.019264,0.006794,0.145159,0.006795,0.028963,0.020764,0.009899,...,0.000820,0.000736,0.001744,0.002201,0.005553,0.004144,0.006101,0.011787,0.001815,2
2,0.006562,0.002379,0.005326,0.009150,0.005789,0.003914,0.018130,0.025686,0.005760,0.008428,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
3,0.019317,0.018023,0.022301,0.032045,0.017454,0.130572,0.049973,0.011506,0.004954,0.039985,...,0.001749,0.000995,0.005361,0.006977,0.001602,0.000675,0.002478,0.004828,0.001521,0
4,0.008251,0.012091,0.010260,0.006741,0.001532,0.013499,0.002521,0.004294,0.001961,0.006768,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2


In [4]:
train_all_df,test_df= train_test_split(df, test_size=0.2, random_state=1)

train_df, val_df = train_test_split(train_all_df,test_size=0.25, random_state=1)

In [5]:
features = df.columns[:192]

In [6]:

X_train= train_df[features]
y_train = train_df['class']

X_val = val_df[features]
y_val = val_df['class']

X_test = test_df[features]
y_test = test_df['class']




In [7]:
import warnings
warnings.filterwarnings('ignore')
best_pars,model =logistic_param_selection(X_train,X_val,y_train,y_val)

In [8]:
model = runBestLogistic(X_train,y_train,X_val,y_val,best_pars)
runBestLogistic(X_train,y_train,X_test,y_test,best_pars)

              precision    recall  f1-score   support

           0       0.53      0.62      0.57        29
           1       0.23      0.27      0.25        11
           2       0.23      0.29      0.26        17
           3       0.67      0.48      0.56        42

   micro avg       0.46      0.46      0.46        99
   macro avg       0.41      0.42      0.41        99
weighted avg       0.50      0.46      0.47        99

Accuracy: 0.46464646464646464
              precision    recall  f1-score   support

           0       0.53      0.55      0.54        31
           1       0.07      0.12      0.09         8
           2       0.29      0.33      0.31        21
           3       0.60      0.45      0.51        40

   micro avg       0.43      0.43      0.43       100
   macro avg       0.37      0.36      0.36       100
weighted avg       0.47      0.43      0.45       100

Accuracy: 0.43


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=2000,
          multi_class='ovr', n_jobs=None, penalty='l2', random_state=None,
          solver='sag', tol=0.0001, verbose=0, warm_start=False)

# Dump four probabilites to disk using leave one out

In [10]:
def runBestLogistic(train,train_labels,test,best_pars):
    
    
    c = best_pars['C']
    multi_class = best_pars['multi_class']
    penalty = best_pars['penalty']
    solver = best_pars['solver']
    

    
    
    clf = LogisticRegression(C=c, multi_class= multi_class,penalty=penalty,solver=solver,
                             class_weight='balanced',max_iter=2000)
    
    
    

    clf.fit(train, train_labels)
    
    predicted=clf.predict_proba([test])
    #print(classification_report(test_labels, predicted))
    #print ("Accuracy: {}".format(accuracy_score(test_labels, predicted)))
    return predicted





In [26]:
df.head()
import time
all_X = df[features]


In [33]:


pred_probs = []
t1 = time.time()

for i, row in all_X.iterrows():
    
    
    
    X_train = pd.concat([all_X.iloc[:i],all_X.iloc[i+1:]])
    y_train = all_y[:i] + all_y[i+1:]
    
    
    
    X_val = row
    #pred_probs.append(runBestLogistic(X_train,y_train,X_val,best_pars))
    p=runBestLogistic(X_train,y_train,X_val,best_pars)
    pred_probs.append(p[0].tolist())
    
t2 = time.time()
print(t2-t1)

49.12250638008118


In [35]:
cls_a = []
cls_b = []
cls_c = []
cls_d = []


for prob in pred_probs:
    cls_a.append(prob[0])
    cls_b.append(prob[1])
    cls_c.append(prob[2])
    cls_d.append(prob[3])

df['emotion_a_prob'] = cls_a 
df['emotion_b_prob'] = cls_b    
df['emotion_c_prob'] = cls_c
df['emotion_d_prob'] = cls_d

In [36]:
df.head()

,E0,E1,E2,E3,E4,E5,E6,E7,E8,E9,...,E187,E188,E189,E190,E191,class,emotion_a_prob,emotion_b_prob,emotion_c_prob,emotion_d_prob
0,0.003607,0.021549,0.024439,0.027935,0.000726,0.066190,0.002535,0.002217,0.003601,0.002111,...,0.001201,0.000567,0.001560,0.003496,0.002619,3,0.248105,0.250254,0.249762,0.251879
1,0.037200,0.014853,0.007491,0.019264,0.006794,0.145159,0.006795,0.028963,0.020764,0.009899,...,0.005553,0.004144,0.006101,0.011787,0.001815,2,0.248510,0.250306,0.249956,0.251228
2,0.006562,0.002379,0.005326,0.009150,0.005789,0.003914,0.018130,0.025686,0.005760,0.008428,...,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.252561,0.249394,0.250055,0.247990
3,0.019317,0.018023,0.022301,0.032045,0.017454,0.130572,0.049973,0.011506,0.004954,0.039985,...,0.001602,0.000675,0.002478,0.004828,0.001521,0,0.248580,0.250107,0.250134,0.251178
4,0.008251,0.012091,0.010260,0.006741,0.001532,0.013499,0.002521,0.004294,0.001961,0.006768,...,0.000000,0.000000,0.000000,0.000000,0.000000,2,0.251402,0.249745,0.250113,0.248741


In [37]:
df.to_csv("Emotion_NewDataSet_with_Probs.csv", encoding='utf-8', index=False)